<a href="https://colab.research.google.com/github/villafue/Machine_Learning_Notes/blob/master/Supervised_Learning/Supervised%20Learning%20with%20Scikit-Learn/Machine%20Learning%20for%20Time%20Series%20Data%20in%20Python/4%20Validating%20and%20Inspecting%20Time%20Series%20Models/4_Validating_and_Inspecting_Time_Series_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>